In [23]:
import pandas as pd
import numpy as np
from davidyu_cfg import *
from finquant.portfolio import build_portfolio
from functions.common.dfcf_fuquan_data import *



def load_hs300_data(start_date,end_date):
    df1 = pd.read_csv("/home/davidyu/stock/data/test/portfolio/hs_300.csv")
    start_date = '2018-01-01'
    end_date = '2020-12-20'
    df2 = df1[(df1["date"]>=start_date)&(df1["date"]<=end_date)]
    df2["dt"]=df2["date"]
    return df2

In [24]:
def cal_daily_return(df1):
    df1 = df1[["dt","close"]].drop_duplicates()
    df_test = df1["close"]
    dates = df1.dt.values
    df_test.index = dates
    df_test.index.name = 'Date'
    # calculate daily returns
    daily_returns = df_test.pct_change().dropna(how="all").replace([np.inf, -np.inf], np.nan)
    return daily_returns
def find_same_date(stock_list,start_date,end_date):
    k=0
  
    for s1 in stock_list:
        if k==0:
            df1 = dfcf_stock_data(s1,start_date,end_date)
            df1 = df1[["dt","close"]].drop_duplicates()
            a = df1["dt"].tolist()
            k+=1
        else:
            df1 = dfcf_stock_data(s1,start_date,end_date)
            df1 = df1[["dt","close"]].drop_duplicates()
            b = df1["dt"].tolist()
            a = list(set(a).intersection(set(b))) 
    return a


def same_date_returns(stock_list,start_date,end_date,same_date):
    df_return_list = []
    stock_index_list = []
    for s1 in stock_list:
        df1 = dfcf_stock_data(s1,start_date,end_date)
        df1 = df1[["dt","close"]].drop_duplicates()
        df_test = df1["close"]
        dates = df1.dt.values
        df_test.index = dates
        df_test.index.name = 'Date'
        # calculate daily returns
        daily_returns = df_test.pct_change().dropna(how="all").replace([np.inf, -np.inf], np.nan)
        # find returns in the same date
        df_return_update = daily_returns[daily_returns.index.isin(same_date)]
        df_return_list.append(df_return_update.values.tolist())
        stock_index_list.append(s1)
        date_index = df_return_update.index.tolist()
    df_out = pd.DataFrame(df_return_list).T
#     df_out.shape
    df_out.index=date_index
#     print(df_out.head(30))
    df_out.columns = stock_index_list
    return df_out 

In [29]:
start_date = '2020-03-01'
end_date = '2020-12-20'
portfolio_stock_list = ["600519","002594","601899","601398"]

df2 = load_hs300_data(start_date,end_date)
df_hs300_return=cal_daily_return(df2)


same_date = find_same_date(portfolio_stock_list,start_date,end_date)
df_portfolio_return = same_date_returns(portfolio_stock_list,start_date,end_date,same_date)
df_hs300_return = df_hs300_return[df_hs300_return.index.isin(same_date)].reset_index()

/home/davidyu/software/Anaconda/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [30]:
port_returns = df_portfolio_return.apply(lambda x:x.mean(),axis =1).reset_index()
port_returns.columns = ["Date","port_return"]
df_m=pd.merge(df_hs300_return,port_returns)
df_m['close'].cov(df_m["port_return"])/df_m['close'].var()


0.9923159688696245